In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

import spacy


import os

In [3]:
#df = pd.read_csv('ira_tweets_csv_hashed.csv', low_memory = False)

In [1]:
# df.columns

In [2]:
# FILTER DOWN TO ENGLISH TWEETS AND A SUBSET OF COLUMNS
# en_tweets = df[df.tweet_language == "en"]

# col_list = ['tweetid', 'userid','tweet_text', 'tweet_time']

# en_tweets = en_tweets.loc[:,col_list]

#### Start here for now

In [2]:
en_tweets = pd.read_csv("/Users/benjaminforleo/Box/spring_project/en_tweets.csv", 
                        low_memory = False)


In [3]:
#en_tweets = en_tweets.drop_duplicates(subset = 'tweet_text')

In [3]:
from spacy.tokens import Token
Token.set_extension('is_hashtag', default = False)
Token.set_extension('is_account', default = False)

##### import functions, preprocess and tokenize the text

from processing_functions import hashtag_pipe
from processing_functions import is_account_pipe


nlp = spacy.load("en", disable = ['parser'])
nlp.add_pipe(hashtag_pipe)
nlp.add_pipe(is_account_pipe)

#en_tweets.tweet_text = en_tweets.tweet_text.apply(preprocessing)

parsed_tweets = list(nlp.pipe(en_tweets.tweet_text[:100]))

In [83]:
from spacy.lang.en.stop_words import STOP_WORDS
import re

link_pattern = re.compile("(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?")

token_list = []

for token in parsed_tweets[17]:
    
    if token._.is_account or token._.is_hashtag:
        token_list.append(token.text)
  
    
    elif (token.text != "RT") and not(token.text in STOP_WORDS) \
    and not bool(re.match(link_pattern, token.text)) \
    and not(token.is_punct):
        token_list.append(token.lemma_)
        

In [78]:
parsed_tweets[17].ents

(Bobby Seale, Huey Newton, the Black Panther Party, 1966)

In [4]:
from processing_functions import clean_doc

In [8]:
test = list(map(clean_doc, parsed_tweets))

In [13]:
from gensim import corpora

dictionary = corpora.Dictionary(test)

print(dictionary)

Dictionary(685 unique tokens: ['apparent', 'as', 'cloud', 'faith', 'make']...)


In [15]:
corpus = [dictionary.doc2bow(text) for text in test]

In [18]:
from gensim import models
tfidf = models.TfidfModel(corpus)

In [20]:
for document in tfidf[corpus]:
    print(document)

[(0, 0.30507286169821896), (1, 0.30507286169821896), (2, 0.30507286169821896), (3, 0.25915482058111233), (4, 0.25915482058111233), (5, 0.30507286169821896), (6, 0.30507286169821896), (7, 0.23229448842103284), (8, 0.30507286169821896), (9, 0.30507286169821896), (10, 0.30507286169821896), (11, 0.25915482058111233)]
[(12, 0.17612086163829657), (13, 0.5683254472852047), (14, 0.5683254472852047), (15, 0.5683254472852047)]
[(12, 0.21551336601676407), (16, 0.6954413520242764), (17, 0.3477206760121382), (18, 0.5907669984320657)]
[(7, 0.38244804888342804), (12, 0.3113012143112473), (17, 0.5022698623494486), (19, 0.5022698623494486), (20, 0.5022698623494486)]
[(18, 0.7071067811865475), (21, 0.7071067811865475)]
[(12, 0.14331267439651546), (22, 0.4624565143526873), (23, 0.4624565143526873), (24, 0.4624565143526873), (25, 0.4624565143526873), (26, 0.35213259816206943)]
[(12, 0.1647236620657547), (27, 0.5315477567571728), (28, 0.4515418473012674), (29, 0.5315477567571728), (30, 0.4515418473012674)]